In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import seaborn as sns

In [ ]:
df1 = pd.read_json("./runtimes/lrz-h100-toxicainment_videos_log.jsonl", lines=True)
df2 = pd.read_json("./runtimes/css-toxicainment_videos_log.jsonl", lines=True)
df = pd.concat([df1, df2], ignore_index=True)
df.head(2)

In [ ]:
import json

with open("./runtimes/runid_2_concurrency.json", "r") as f:
    data = json.load(f)

rows = []
for group_data in data:
    for uid, concurrency in group_data["concurrency"].items():
        rows.append(
            {"Run_ID": uid, "concurrency": concurrency, "name": group_data["name"]}
        )
concurrency_df = pd.DataFrame(rows)
concurrency_df.head(2)

In [5]:
begin = df.groupby("Run_ID", as_index=False).agg(
    t0=("Timestamp", "first"), d0=("Total_Runtime", "first")
)
df = pd.merge(df, begin, on="Run_ID", how="left").assign(
    tx=lambda df: (df["Timestamp"] - df["t0"]).dt.total_seconds() + df["d0"]
)

In [ ]:
df = pd.merge(df, concurrency_df, on="Run_ID", how="left")
df.head(2)

In [ ]:
sdf = df.groupby(["name", "concurrency"], as_index=False).agg(
    avg_runtime=("Total_Runtime", "mean"),
    total_runtime=("Total_Runtime", "sum"),
    first_response=("Total_Runtime", "first"),
    n_requests=("Total_Runtime", "count"),
)
sdf

In [ ]:
ax = sns.scatterplot(sdf, x="concurrency", y="avg_runtime", hue="name")
ax.grid(alpha=0.5)
ax.set_ylim(0, None)

In [ ]:
df["cumsum_runtime"] = df.groupby("Run_ID")["Total_Runtime"].cumsum()
df["id_in_run"] = df.groupby("Run_ID").cumcount()
sns.relplot(
    data=df.query("name == 'lrz-h100'"),
    x="cumsum_runtime",
    y="id_in_run",
    hue="concurrency",
    palette="tab10",  # Using a discrete color palette
    hue_norm=None,    # Ensures discrete colors
    row="name",
    kind="line",
    height=4,
    aspect=2
)